# Marine Heat Wave analysis
The purpose of this notebook is to take in SST data, along with previously computed climatology and threshold, process these and output all the MHW events that occur in the data

First, we import the data and required modules. 

In [ ]:
import xarray as xr
import pandas as pd
import now
import numpy as np
import dask as da
import scipy.ndimage as ndimage
import time
import cartopy.crs as ccrs
import dask_image.ndmeasure
from datetime import date
from matplotlib import pyplot as plt
from dask.distributed import LocalCluster, Client
from cartopy import config
%pylab inline

In [ ]:
plt.rcParams["figure.figsize"] = [10,5]
cluster = LocalCluster(processes=False, local_dir= "/g/data1a/e14/rm6294/dask-workers")
client = Client(cluster)
client

In [ ]:
# Opens Climatology and MHW SST threshold files, that have been created before.
# Note: p stands for past, f for future
localDir = "/g/data1a/e14/rm6294/NOWMHW/"
pThresh = xr.open_dataarray(localDir + 'pastThreshRep.nc')
pClim = xr.open_dataarray(localDir + 'pastClimRep.nc')
fThresh = xr.open_dataarray(localDir + 'futureThreshRep.nc')
fClim = xr.open_dataarray(localDir + 'futureClimRep.nc')

In [ ]:
# Opens NOW SST data, from which we need the SST.
cfg = 'present_vs_future.ini'
sst = now.io.open_nemo_griddata_from_zarr(cfg, grid='T')['tos'] #tos means sst
# SST contains both future and past run. We now split that up.
sstPast = sst.sel(simulation = 'Present')
sstFut = sst.sel(simulation = 'Future')

In [ ]:
# NOW data is daily, and runs from 1990/01/01T12:00:00 to 2008/12/31T12:00:00. We use this to create our t vector.
#Two lines below taken from example_synthetic file in MHW github
t = np.arange(date(1990,1,1).toordinal(),date(2008,12,31).toordinal()+1)
dates = [date.fromordinal(tt.astype(int)) for tt in t]

## Looking for MHW
The method we will be using to identify MHW is to:
* Create a mask on top of the data, which has a binary True/False representing whether the sst at the location is above the corressponding threshold. 
    * Using the mask, try and find consecutive days where the threshold is crossed, as these represent the MHW.
* Once we have all the SSTs that cross the threshold, we will the apply the duration requirement, and look for SSTs that cross the threshold for 5 consecutive days

### Creating a Mask (Crossing the threshold)
We are looking for SST to cross the threshold. We will do this by:
* Subtracting the background climatology from both the threshold and sst, giving us theshold anomaly (thresha) and SST anomaly (ssta).
* Dividing ssta by thresha.
* Saving the severity data, so we can simply call it for use in the animation section.
This process will not only give us a matrix of threshold crosses (as ssta > thresha at this points, ssta/thresa > 1), but also tell us about the severity of the MHW, as temperatures 2 thresholds away from the background will be represented with a two in the matrix and so on.

##### Note: Current looking at Future SSTs compared to a future Clim. The threshold is created by calculating the difference between the past percentile threshold and climatology & then adding that to the future Clim (done elsewhere).

In [ ]:
%%time
# Finding thresha and ssta
thresha = fThresh.chunk({'x':50, 'y':50, 'time_counter':365}) - fClim.chunk({'x':50, 'y':50, 'time_counter':365})
ssta = sstFut.chunk({'x':50, 'y':50, 'time_counter':365}) - fClim.chunk({'x':50, 'y':50, 'time_counter':365})

In [ ]:
%%time
mhws = xr.where(ssta > thresha, ssta/thresha, 0)

# The next line removes any points that were made very large due to epsilon error in the ssta/thresha div
mhws = xr.where(mhws > 10, 0, mhws)

mhws['nav_lon'] = mhws['nav_lon'] % 360 

In [ ]:
# Saving the severity data. This is only the halfway point in the process - with duration still to go.
mhws.chunk({'x': 431, 'y':289, 'time_counter':6940}).to_netcdf("/g/data1a/e14/rm6294/NOWMHW/severityMHWNOW_futRun.nc")

In [ ]:
ax = plt.subplot(111, projection=ccrs.PlateCarree(central_longitude=0))
now.plot.add_map(lon_min = 90, lon_max = 180, lat_min = -55,ax = ax, lat_max = 5)
mhws.isel(time_counter = 2).plot(x='nav_lon', y='nav_lat', cmap = 'hot_r',vmin = 0, vmax = 4.5, ax = ax);
#plt.savefig("/g/data1a/e14/rm6294/NOWMHW/MHWPlots/" + 'mhwPlot_{:0>3d}.png'.format(i),bbox_inches='tight')

## Checking for duration
We now have all the SSTs that cross our threshold. This section will aim to find the events that meet the 5 consecutive days with at most a 2 day gap criterion. To do this we will:
* For each time series, track how long each MHW occurs for.
* Eliminate those that are less than 5 days long 

In [ ]:
# Starting with looking at future MHWs
# We are starting with the saved mhw files, which have been saved after the above mask and comparison procedure was 
# executed
localDir = "/g/data1a/e14/rm6294/NOWMHW/"
mhws = xr.open_dataarray(localDir + 'severityMHWNOW_futRun.nc')

In [ ]:
chunk_size = 10

In [ ]:
mhws = mhws.chunk({'x': chunk_size, 'y': chunk_size})

In [ ]:
mhws

In [ ]:
def day2ns(days):
    # Converts days to nanoseconds. days -> hours -> seconds -> nanoseconds
    return int(days * 24 * 3600 * 1e9)

In [ ]:
struc = np.zeros((3,3,3))
# Just looking for consecutive SST threshold crosses in the time dimension:
struc[:,1,1] = 1
labeled_array, num_features = dask_image.ndmeasure.label(mhws.data, structure = struc)

In [ ]:
%%time
labeled_array = labeled_array.compute()
num_features = num_features.compute()

In [ ]:
# Converting from date-time to nanoseconds
mhw_dates = pd.to_numeric(mhws['time_counter'])

# Creates mhw_dur object which has a time series at each grid point
mhw_dur = xr.broadcast(mhws, mhw_dates)[1]

In [ ]:
lbl = xr.full_like(mhws, labeled_array)
lbl = lbl.chunk({'x' : 50,'y' : 50})

In [ ]:
# Sets all points in mhw_dur where SST didn't cross threshold to 0. Might be useless, as max and min functions should 
# work regardless
mhw_dur = xr.where(lbl > 0, mhw_dur, 0)  

In [ ]:
%%time
maxMHW = dask_image.ndmeasure.maximum(mhw_dur, labels=lbl)

In [ ]:
%%time
minMHW = dask_image.ndmeasure.minimum(mhw_dur, labels=lbl)

In [ ]:
maxMHW

In [ ]:
# Calculates the length of all potential MHW events.
dur = maxMHW - minMHW + day2ns(1) 

# converting minDur (default = 5) days to nanoseconds
minDur = 5 # default ^
minDur_ns = day2ns(minDur) 
# Finds the event numbers of MHWs with duration >= minDur_ns. 
failed_ev_num = da.array.where(dur < minDur_ns)[0]  + 1 # Adding 1 so that event number matches with value in labeled_array. 

In [ ]:
%%time
failed_ev_num = failed_ev_num.compute()

In [ ]:
%%time
lbl = lbl.compute()

In [ ]:
%%time
# objs has the location of each labeled event 
#i.e objs[0] is the location of the event 1, objs[10] location of event 11  
objs = ndimage.find_objects(lbl)

# failed_objs has the location for all the failed events in labeled_array 
failed_objs = [objs[i - 1] for i in failed_ev_num]

In [ ]:
for x in failed_objs:
    labeled_array[x] = 0

In [ ]:
mhw_preGap = xr.where(lbl > 0, mhwSub, 0)

# Trials

#### Here, we try and try a solution to implement the duration definiton for MHWs. 
We will use a subset of the dataset to do this. Our approach will consider the time series at each grid point and calculate the duration for of consecutive MHW events. If the duration is less than 5 days, we will set it to zero.

In [ ]:
localDir = "/g/data1a/e14/rm6294/NOWMHW/"
mhws = xr.open_dataarray(localDir + 'severityMHWNOW_futRun.nc')
# Creating subset
mhwSub = mhws.isel(x = slice(170,270),y = slice(0,100))

In [ ]:
chunk_size = 20

In [ ]:
mhwSub = mhwSub.chunk({'x': chunk_size, 'y': chunk_size})
mhwSub

In [ ]:
# Plot shows approximately what area we are considering.
ax = plt.subplot(111, projection=ccrs.PlateCarree(central_longitude=0))
now.plot.add_map(lon_min = 90, lon_max = 180, lat_min = -55,ax = ax, lat_max = 4.5)
mhwSub.isel(time_counter = 6935).plot(x = 'nav_lon', y = 'nav_lat', cmap = 'hot_r', ax = ax);

#### Date approach

In [58]:
def day2ns(days):
    # Converts days to nanoseconds. Days -> Hours -> Seconds -> nanoseconds
    return int(days * 24 * 3600 * 1e9)

In [59]:
%%time
struc = np.zeros((3,3,3))
# Just looking for consecutive SST threshold crosses in the time dimension:
struc[:,1,1] = 1
labeled_array, num_features = dask_image.ndmeasure.label(mhwSub.data, structure = struc)
#ndimage.label(mhwSub.data, structure = struc)

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 3.54 ms


/g/data1/e14/rm6294/miniconda3/envs/now-postprocess/lib/python3.6/site-packages/dask_image/ndmeasure/__init__.py:214: RuntimeWarning: ``input`` does not have 1 chunk in all dimensions; it will be consolidated first
  warn("``input`` does not have 1 chunk in all dimensions; it will be consolidated first", RuntimeWarning)


In [60]:
# Creating an xarray object with the same coordinates as mhws for labeled_array
lbl = xr.full_like(mhwSub, labeled_array)

In [61]:
# Converting from date-time to nanoseconds
mhw_dates = pd.to_numeric(mhwSub['time_counter'])

# Creates mhw_dur object which has a time series (in nanoseconds) at each grid point
mhw_dur = xr.broadcast(mhwSub, mhw_dates)[1]

In [62]:
# Sets all points in mhw_dur where SST didn't cross threshold to 0. Might be useless, as we don't actually use mhw_dur
mhw_dur = xr.where(lbl > 0, mhw_dur, 0)  

In [121]:
indexs = da.array.from_array(da.array.arange(1, 1 + num_features), chunks = 6940)

In [122]:
indexs

dask.array<array, shape=(885142,), dtype=int64, chunksize=(6940,)>

In [114]:
print("mhw_dur: type =", type(mhw_dur.data), ", chunks =", mhw_dur.data.chunksize)
print("    lbl: type =", type(lbl.data), ", chunks =", lbl.data.chunksize)
print(" mhwSub: type =", type(mhwSub.data), ", chunks =", mhwSub.data.chunksize)

mhw_dur: type = <class 'dask.array.core.Array'> , chunks = (6940, 20, 20)
    lbl: type = <class 'dask.array.core.Array'> , chunks = (6940, 20, 20)
 mhwSub: type = <class 'dask.array.core.Array'> , chunks = (6940, 20, 20)


In [123]:
# Calculates the max and min time points for all labeled points. I.e. the beggining and end dates for SST events 
maxMHW = dask_image.ndmeasure.maximum(mhw_dur, labels=lbl, index=indexs)
#minMHW = dask_image.ndmeasure.minimum(mhw_dur, labels=lbl, index=indexs)

KeyboardInterrupt: 

In [ ]:
# Calculates the length of all potential MHW events.
dur = maxMHW - minMHW + day2ns(1) 

In [ ]:
# converting minDur (default = 5) days to nanoseconds
minDur = 5 # default ^
minDur_ns = day2ns(minDur) 
# Finds the event numbers of MHWs with duration >= minDur_ns. 
failed_ev_num = da.array.where(dur < minDur_ns)[0]  + 1 # Adding 1 so that event number matches with value in labeled_array. 

In [ ]:
maxMHW.compute()

In [ ]:
failed_ev_num.compute()

In [ ]:
# objs has the location of each labeled event 
#i.e objs[0] is the location of the event 1, objs[10] location of event 11  
objs = ndimage.find_objects(lbl.astype('int'))

In [ ]:
# failed_objs has the location for all the failed events in labeled_array 
failed_objs = [objs[i - 1] for i in failed_ev_num]

In [ ]:
for loc in failed_objs:
    lbl.data[loc] = 0

In [ ]:
%%time
mhw_preGap = xr.where(lbl > 0, mhwSub, 0)

In [ ]:
# Checks if there are the right amount of locations
print(len(failed_ev_num))
len(failed_objs) == len(failed_ev_num) 

In [ ]:
copy = xr.open_dataarray(localDir + 'severityMHWNOW_futRun.nc').isel(x = slice(170,270),y = slice(0,100))
#copy = copy.chunk({'x': 50, 'y': 50})

In [ ]:
#f, (ax1, ax2) = plt.subplots(1, 2)
ax1 = plt.subplot(211, projection=ccrs.PlateCarree(central_longitude=0))
ax2 = plt.subplot(212, projection=ccrs.PlateCarree(central_longitude=0))

now.plot.add_map(lon_min = 90, lon_max = 180, lat_min = -55,ax = ax1, lat_max = 4.5)
mhw_preGap.isel(time_counter = 6935).plot(x = 'nav_lon', y = 'nav_lat', cmap = 'hot_r', ax = ax1);

now.plot.add_map(lon_min = 90, lon_max = 180, lat_min = -55,ax = ax2, lat_max = 4.5)
copy.isel(time_counter = 6935).plot(x = 'nav_lon', y = 'nav_lat', cmap = 'hot_r', ax = ax2);

In [ ]:
copy.isel(x = 1, y = 1).plot()
mhw_preGap.isel(x = 1, y = 1).plot();

In [ ]:
# Check to see if something has actually changed
(mhw_preGap != copy).sum().compute()

#### Now need to calculate gaps

In [ ]:
ev_nums = np.where(dur >= minDur_ns)[0]  + 1

In [ ]:
ev_nums[0:20] - 1 #<- index when counting from 0 

In [ ]:
# Shows the maximum and minimum date for the <index> MHW 
index = 73
print(pd.to_datetime(minMHW[index], unit = 'ns'), pd.to_datetime(maxMHW[index], unit = 'ns'), pd.to_timedelta(dur[index]))

In [ ]:
# This operation will only leave the dates in mhw_dur during which MHWs occured
mhw_dur = xr.where(labeled_array > 0, mhw_dur, 0)  

In [ ]:
lbls, ne = ndimage.label(labeled_array, structure = struc)

In [ ]:
np.where(lbls == 1)
# Maybe try find where the events are,and see if there is an event in 2 times_steps.

In [ ]:
#pd.to_datetime(maxs, unit = 'ns')

#### Cobbled Together

In [ ]:
def whereIs(array, ev_num):
    # Finds location of event <ev_num> in array
    return np.where(array == ev_num)[0]

In [ ]:
def clear(array, loc):
    # Removes SST events in array at specified location loc
    array[loc] = 0
    return array

In [ ]:
def duration(array, num_events):
    # Takes in a time-series and implements the 5 day requirement for it.
    min_duration = 5
    for ev in range(1, num_events + 1):
        loc = whereIs(array, ev)
        if len(loc) < min_duration:
            array = clear(array, loc)
    return array

In [ ]:
struc = np.zeros((3,3,3))
struc[:,1,1] = 1
labeled_array1, num_features1 = ndimage.label(mhws.data, structure = struc)

In [ ]:
%%time
labeled_array, num_features = da.array.apply_along_axis(ndimage.label, mhws.get_axis_num('time_counter'), mhws.data).compute()

In [ ]:
# Storing number of events as an integer
numEv = xr.DataArray(num_features.astype('int'), dims = ('x', 'y'))

In [ ]:
%%time
for x in range(0, 430):
    for y in range(0, 288):
        duration(labeled_array[y,x], numEv.isel(x = x, y = y).data)

In [ ]:
mhws_dur = mhws_dur.chunk({'x' : 50, 'y'  : 50})

In [ ]:
mhws_dur = xr.full_like(mhws, labeled_array1)

In [ ]:
labeled_array, num_features = da.array.apply_along_axis(ndimage.label, mhwSub.get_axis_num('time_counter'), mhwSub.data)

In [ ]:
%%time
labeled_array = labeled_array.compute()
num_features = num_features.compute()

In [ ]:
# Checking if we can store labeled_array as an xarray object
numEv = xr.DataArray(num_features.astype('int'), dims = ('x', 'y'))

In [ ]:
%%time
for x in range(0, 50):
    for y in range(0, 50):
        duration(labeled_array[x,y], numEv.isel(x = x, y = y).data)

In [ ]:
l,ne = ndimage.label(mhwSub.isel(x = 0, y = 0))

In [ ]:
ne

In [ ]:
plot(l, label = 'all')
plot(labeled_array[0,0], label = 'real')
plt.legend();

plt.xlim(1000,2000)

In [ ]:
def whereIs(array, ev_num):
    # Finds location of event <ev_num> in array
    return np.where(array == ev_num)[0]

In [ ]:
def check(array, loc):
    # Removes SST events in array at specified location loc
    array[loc] = 0
    return array

In [ ]:
def duration(array, num_events):
    # Takes in a time-series and implements the 5 day requirement for it.
    min_duration = 5
    for ev in range(1, num_events + 1):
        loc = whereIs(array, ev)
        if len(loc) < min_duration:
            array = check(array, loc)
    return array

    '''
       # Maybe make it recursive?? 
    
    duration(array, num_events, index)
    if index < num_events:
        loc = da.delayed(whereIs)(array, ev)
        array = da.delayed(check)(array, loc)
        da.delayed(duration)(array, num_events, index + 1)
    else:
        return array
    '''

In [ ]:
checked[:,0,0]  = 0

In [ ]:
labeled_array, num_features = da.array.apply_along_axis(ndimage.label, mhwSub.get_axis_num('time_counter'), mhwSub.data)
checked = (np.zeros_like(mhwSub)).astype('object')


for x in range(0, 50):
    for y in range(0, 50):
        checked[:,x,y] = da.delayed(duration, pure = True)(labeled_array[x,y], num_features[x,y], 0)


In [ ]:
plot(a)
plot(labeled_array[0,0]);

In [ ]:
'''
def check(length, min_duration):
    # Checks if length of event is greater than the min duration required
    return (length < min_duration)

def clear(array, loc):
    # Removes SST events in array at specified location loc
    array[loc] = 0
    return array
'''

In [ ]:
np.where(calc[0,0] == num_features[0,0])

In [ ]:
# Sample for how the algorigthm will look. Problem is, we have to iterate i & j & num_events. Gonna look into 
# dask.delayed to try and loop through this.
new = []
i = 0; j = 0; num_events = num_features[i,j];
for ev in range(1, num_events):
    loc = da.delayed(whereIs)(calc[i,j], ev)
    new = da.delayed(check)(calc[i,j], loc)

In [ ]:
num_features = num_features.astype('int')

In [ ]:
%%time

'''
Currently: This algorithm finds events with duration less than 5, and sets the values to 0.
It has the capability to identify events that will occur two days later, but can't yet link them up. 
Additionally, there is no parralelisation capability - which is the whole point of this thing. Might be worht
looking at the percentile application to look into parralelisation.
'''


a = num_features
for i in range(1,num_features):
    loc = np.where(labeled_array == i)
    dur = loc[0][-1] - loc[0][0] + 1
    if dur < 5:
        a -= 1
        labeled_array[loc] = 0
    elif labeled_array[loc[0][-1] + 1, loc[1][-1], loc[2][-1]] != 0:
        print('Conection!')
        print(i)
        
print('Found', num_features - a, 'false events')

### Trying to spatially connect MHWs and then calculate the duration connected events.  
Currently links events that are spatially and/or temporally connected, but doesn't change original dataset.

In [ ]:
%%time
mhwSub = mhwSub.chunk({'time_counter':10})
# Structure such that diagonal connections in both spatial and temporal dimensions are counted. 
struc = np.ones((3,3,3)) 
# This function finds strings of consecutive nonzero numbers. 
labeled_array, num_features = ndimage.label(mhwSub, struc)

In [ ]:
num_features

In [ ]:
%%time
# Brute force approach. Looping through each of the labelled events, and calculating their duration
feat = num_features
for i in range(1, num_features):
    time = np.where(labeled_array == i)[0]
    dur = time[-1] - time[0] + 1
    if dur < 5:
        feat -= 1
        
print("# MHWs originally:", num_features, "\n # MHWs after duration constraint: ", feat)

In [ ]:
# Get the time record and broadcast its shape to be 3D using xarray.broadcast
time_record = xr.broadcast(mhwSub.isel(x=1,y=1),mhwSub.isel(x=1,y=1), mhwSub.isel(x=1,y=1))

In [ ]:
 b

In [ ]:
np.where(labeled_array == 11)
#print(len(I))
#print(I,J,K, sep = '\n')

In [ ]:
%%time
I, J, K = np.where(labeled_array == 9)

labeled_array[(I,J,K)] = 0
print(labeled_array[(I,J,K)])#.isel(time_counter = 2).plot()

In [ ]:
index = 14
matshow(mhwSub[index], origin = 'lower');
matshow(labeled_array[index],origin='lower');

In [ ]:
maxs = ndimage.maximum(mhwSub.isel(x=0,y=0), labels = labeled_array[:,0,0], index=np.arange(1, num_features + 1))
mins = ndimage.minimum(mhwSub.isel(x=0,y=0), labels = labeled_array[:,0,0], index=np.arange(1, num_features + 1))

In [ ]:
#mhwSub.isel(x=0,y=0).plot()
plot(maxs)
plot(mins)
xlim(3800,3840);

In [ ]:
maxs = pd.to_datetime(maxs, unit = 'ns')
mins = pd.to_datetime(mins, unit = 'ns')
plot(maxs)
plot(mins)

In [ ]:
dif = maxs - mins

In [ ]:
dif

In [ ]:
matshow(labeled_array[0], origin = 'lower');

In [ ]:
index = 3824
print(maxs[index], mins[index])

In [ ]:
shape(mins)

In [ ]:
durations = maxs - mins

In [ ]:
print('1', durations[2])

#### First, we will try with a subset of the entire dataset.
This subset will focus on the past data.

In [ ]:
# Creating subset:
# key: p = past, f = fast, c = clim, t = threshold 
pcSub = pClim.isel(x = slice(220,320),y = slice(50,150))
ptSub = pThresh.isel(x = slice(220,320),y = slice(50,150))
sstSub = sstPast.isel(x = slice(220,320),y = slice(50,150))

In [ ]:
# Showing that we have taken a subset of the spacial dimension
sstSub.isel(time_counter = 1).plot();

In [ ]:
pClim

In [ ]:
%%time
# Finding thresha and ssta
thresha = ptSub.chunk({'x':10, 'y':10, 'time_counter':365}) - pcSub.chunk({'x':10, 'y':10, 'time_counter':365})
ssta = sstSub.chunk({'x':10, 'y':10, 'time_counter':365}) - pcSub.chunk({'x':10, 'y':10, 'time_counter':365})

In [ ]:
ssta.isel(x=1,y=2).plot(size = 10)
thresha.isel(x=1,y=2).plot();

In [ ]:
%%time
#mhws = da.array.apply_along_axis(np.where, ssta.get_axis_num('time_counter'), ssta > thresha, ssta / thresha, 0)
#dif = ssta - thresha
mhws = xr.where(ssta > thresha, ssta/thresha, 0)

This mhws array represents all the values for which the threshold is exceeded. MHWs have the additional stipulation that they must last for 5 or more days. The next part tries to follow what Oliver did to combat this problem

In [ ]:
# Here we create the array structure that the label function will use
s = generate_binary_structure(2,2)
struc = np.array([s,s,s])
struc[1][1][1] = 0

In [ ]:
struc

In [ ]:
%%time
# This function finds strings of consecutive nonzero numbers. 
labeled_array, num_features = label(mhws,struc)

In [ ]:
num_features

In [ ]:
%%time
i = 0
for ev in range(1,num_features+1):
    event_duration = (labeled_array == ev).sum()
    if event_duration < 5:
        i += 1
        #print("Event " + str(ev) + " is a " + str(event_duration) + " day event ")
        labeled_array[np.where(labeled_array == ev)] = [0]
        
num_features -= i

In [ ]:
marHw = xr.DataArray(labeled_array, coords = mhws.coords, dims = mhws.dims)

In [ ]:
pcSub.isel(x=2, y = 60, time_counter= 1)

In [ ]:
pcSub.isel(time_counter = 4225).plot();

In [ ]:
mhws.isel(time_counter = 90).plot(cmap = 'hot_r', vmin = 0, vmax = 4);

In [ ]:
matshow(labeled_array[90], origin = 'lower')#.isel(time_counter = 1).plot(cmap = 'hot_r', vmin = 0, vmax = 4);